In [64]:
import json
import os
from unskript import nbparams
from unskript.secrets import ENV_MODE, ENV_MODE_AWS
from unskript.fwk.workflow import Task, Workflow

env = {"ENV_MODE":"ENV_MODE_AWS","TENANT_ID":"982dba5f-d9df-48ae-a5bf-ec1fc94d4882","PROXY_ID":"1499f27c-6406-4fbd-bd1b-c6f92800018f","TENANT_URL":"https://tenant-staging.alpha.unskript.io","AWS_REGION":"us-west-2"}
secret_store_cfg = {"SECRET_STORE_TYPE":"SECRET_STORE_TYPE_AWS","AWS_SECRET_PREFIX":"staging","AWS_REGION":"us-west-2"}
os.environ["UNSKRIPT_REDIS_HOST"] = "redis-master.redis.svc.cluster.local"
os.environ["UNSKRIPT_TOKEN"] = "5c4a5754-0600-11ec-9a03-0242ac130003"
os.environ["TENANT_URL"] = env["TENANT_URL"]
paramDict = {"Region": "us-west-2", "Tag_Key": "Description", "Tag_Value": "Unskript"}
paramDict.update(env)
paramDict.update(secret_store_cfg)
paramsJson = json.dumps(paramDict)
nbParamsObj = nbparams.NBParams(paramsJson)
Region = nbParamsObj.get('Region')
Tag_Key = nbParamsObj.get('Tag_Key')
Tag_Value = nbParamsObj.get('Tag_Value')

w = Workflow(env, secret_store_cfg, None, global_vars=globals())

<img src="https://unskript.com/wp-content/uploads/2022/10/unSkript-Logo.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to build a process around Mandatory Tags Across All AWS Resources using unSkript legos.</b>
</div>

<br>

<center><h2>Enforce Mandatory Tags Across All AWS Resources</h2></center>

# Steps Overview
   1. **AWS Get Untagged Resources**: List all the Untagged Resources instanceIDs in the given region. This will also return data about the instances to help identify who the owners are.
   2. **Get tag keys of all Resources** This is a similar list to #1, but includes all instances, and all of the tages for each instance.
   3. AWS Attach Tags to Resources: This action takes in an instance ID and a tag key:value pair.  Run this action as many times as needed to fully tag your instances.
   4. compare tag keys against required list: This final Action looks through all actions, and compares the Tag Keys with the required tag list.  If an instance is not in compliance, it is exported.
   
 The eventual goal is that after all required instances are labelled in step 3, step 4 will have only instances that are no longer needed, and can be removed from AWS.

Here we will use unSkript AWS Get Untagged Resources Lego. This lego take region: str as input. This inputs is used to find out all Untagged Resources.

In [24]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
import pprint

from beartype import beartype
@beartype
def aws_get_untagged_resources_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_get_untagged_resources(handle, region: str) -> List:
 
    print("region",region)
    ec2Client = handle.client('ec2', region_name=region)
    #res = aws_get_paginator(ec2Client, "describe_instances", "Reservations")
    res = aws_get_paginator(ec2Client, "describe_instances", "Reservations")
    result = []
    for reservation in res:
        for instance in reservation['Instances']:       
            try:
                #has tags
                tagged_instance = instance['Tags']
            except Exception as e:
                #no tags
                result.append({"instance":instance['InstanceId'],"type":instance['InstanceType'],"imageId":instance['ImageId'], "launched":instance['LaunchTime'] })
    return result


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "dougsaws",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "32a18484-b86d-4f04-ac6b-9d1d45127d82"
}''')
task.configure(inputParamsJson='''{
    "region": "Region"
    }''')
task.configure(outputName="UntaggedResources")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_untagged_resources, lego_printer=aws_get_untagged_resources_printer, hdl=hdl, args=args)

Here we will use unSkript Get Tag Keys Of All Resources Lego. This lego take region: str as input. This input is used to find out all Tag Keys of Resources.

In [65]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
import pprint

from beartype import beartype
@beartype
def aws_resources_tags_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_resources_tags(handle, region: str) -> List:
    """aws_resources_tags Returns an List of all Resources Tags.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: str
        :param region: Region to filter resources.

        :rtype: List of all Resources Tags.
    """
    ec2Client = handle.client('ec2', region_name=region)
    #res = aws_get_paginator(ec2Client, "describe_instances", "Reservations")
    res = aws_get_paginator(ec2Client, "describe_instances", "Reservations")
    result = []
    for reservation in res:
        for instance in reservation['Instances']:       
            try:
                #has tags
                tagged_instance = instance['Tags']
                result.append({"instance":instance['InstanceId'],"type":instance['InstanceType'],"imageId":instance['ImageId'], "launched":instance['LaunchTime'], "tags": tagged_instance})
            except Exception as e:
                #no tags
                result.append({"instance":instance['InstanceId'],"type":instance['InstanceType'],"imageId":instance['ImageId'], "launched":instance['LaunchTime'], "tags": []})
    return result


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "dougsaws",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "32a18484-b86d-4f04-ac6b-9d1d45127d82"
}''')
task.configure(inputParamsJson='''{
    "region": "Region"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_resources_tags, lego_printer=aws_resources_tags_printer, hdl=hdl, args=args)

Here we will use unSkript AWS Attach Tags to Resources Lego. This lego take handle, resource_arn: list, tag_key: str, tag_value: str, region: str as input. This input is used to attach mandatory tags to all untagged Resources.

In [52]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
import pprint

from beartype import beartype
@beartype
def aws_tag_resources_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_tag_resources(handle, instanceId: str, tag_key: str, tag_value: str, region: str) -> Dict:

    ec2Client = handle.client('ec2', region_name=region)
    result = {}
    try:
        response = ec2Client.create_tags(
            Resources=[
                instanceId
            ],
            Tags=[
                {
                    'Key': tag_key,
                    'Value': tag_value
                },
            ]
        )
        result = response

    except Exception as error:
        result["error"] = error

    return result






task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "dougsaws",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "32a18484-b86d-4f04-ac6b-9d1d45127d82"
}''')
task.configure(continueOnError=False)
task.configure(inputParamsJson='''{
    "region": "Region",
    "tag_key": "\\"Environment\\"",
    "tag_value": "\\"test\\"",
    "instanceId": "\\"i-0ec9048cb5520b225\\""
    }''')
task.configure(iterJson='''{
    "iter_enabled": false,
    "iter_list_is_const": false,
    "iter_list": "UntaggedResources[",
    "iter_parameter": "resource_arn"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_tag_resources, lego_printer=aws_tag_resources_printer, hdl=hdl, args=args)

In [63]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
import pprint

from beartype import beartype
@beartype
def aws_get_resources_out_of_compliance_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_get_resources_out_of_compliance(handle, region: str, requiredTags: list) -> List:

    ec2Client = handle.client('ec2', region_name=region)
    #res = aws_get_paginator(ec2Client, "describe_instances", "Reservations")
    res = aws_get_paginator(ec2Client, "describe_instances", "Reservations")
    result = []
    for reservation in res:
        for instance in reservation['Instances']:       
            try:
                #has tags
                allTags = True
                keyList = []
                tagged_instance = instance['Tags']
                #print(tagged_instance)
                #get all the keys for the instance
                for kv in tagged_instance:
                    key = kv["Key"]
                    keyList.append(key)
                #see if the required tags are represented in the keylist
                #if they are not - the instance is not in compliance
                for required in requiredTags:
                        if required not in keyList:
                            allTags = False
                if not allTags:
                    # instance is not in compliance
                    result.append({"instance":instance['InstanceId'],"type":instance['InstanceType'],"imageId":instance['ImageId'], "launched":instance['LaunchTime'], "tags": tagged_instance})
                
            except Exception as e:
                #no tags
                result.append({"instance":instance['InstanceId'],"type":instance['InstanceType'],"imageId":instance['ImageId'], "launched":instance['LaunchTime'], "tags": []})
    return result



task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "dougsaws",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "32a18484-b86d-4f04-ac6b-9d1d45127d82"
}''')
task.configure(inputParamsJson='''{
    "region": "\\"us-west-2\\"",
    "requiredTags": "[\\"CostCenter\\", \\"Environment\\"]"
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_resources_out_of_compliance, lego_printer=aws_get_resources_out_of_compliance_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS legos to attach tags. This Runbook gets the list of all untagged resources of a given region, discovers tag keys of the given region and attaches mandatory tags to all the untagged resource. To view the full platform capabilities of unSkript please visit https://unskript.com